In [10]:
import os
import shutil
import warnings
import DICOMLib
import ctk
warnings.filterwarnings('error')
class TemporaryDatabase(ctk.ctkDICOMDatabase):
    @property
    def directory(self):
        return self._directory
    def __init__(self, path):
        ctk.ctkDICOMDatabase.__init__(self)
        self._directory = path
        self.reset()
        self.openDatabase(os.path.join(self.directory, "ctkDICOM.sql"), "mpReviewPreprocessor")
        self.initializeDatabase()
    def reset(self):
        if os.access(self.directory, os.F_OK):
            shutil.rmtree(self.directory)
            os.mkdir(self.directory)

In [11]:
import sys
cw = os.getcwd()
sys.path.append(cw)
sys.path.append(os.path.dirname(cw))
# import utils.settings as S

In [12]:
dicom_root = os.path.join("/home/sedghi/Desktop/PROSTATE-DIAGNOSIS")
output_folder = os.path.join("/home/sedghi/Desktop/PROSTATE-DIAGNOSIS_nrrd")
patient_folders = os.listdir(dicom_root)
T2_not_desired = ['sT2','COR','SAG','Pelvis']
T2_not_desired = []

In [13]:
if not os.path.exists(output_folder):
    os.makedirs(output_folder)

In [14]:

erronous_patients_t2 = []
for patient_folder in sorted(patient_folders, reverse=True):
    print('-' * 100)
    print(patient_folder)
    print('-' * 100)
    patient_folder_path = os.path.join(dicom_root, patient_folder)
    output_patient_folder = os.path.join(output_folder, patient_folder)
    if not os.path.isdir(output_patient_folder):
        os.mkdir(output_patient_folder)
    dicomDatabase = TemporaryDatabase('/home/sedghi/tmp')
    indexer = ctk.ctkDICOMIndexer()
    indexer.addDirectory(dicomDatabase, patient_folder_path)
    patients = dicomDatabase.patients()
    for patient in patients:
        for study in dicomDatabase.studiesForPatient(patient):
            output_study_folder = os.path.join(output_patient_folder, study)
            if not os.path.isdir(output_study_folder):
                os.mkdir(output_study_folder)
            series = dicomDatabase.seriesForStudy(study)
            for seriesIndex, currentSeries in enumerate(series, start=1):
                try:
                    filenames = dicomDatabase.filesForSeries(currentSeries)
                    series_description = dicomDatabase.fileValue(filenames[0], "0008,103E")
                    series_number = dicomDatabase.fileValue(filenames[0], "0020,0011")
                    if ('T2W_TSE_AX' in str(series_description)) & (not any(item.upper() in str(series_description).upper() for item in T2_not_desired)):
                        plugin_name = 'DICOMScalarVolumePlugin'
                        plugin = slicer.modules.dicomPlugins[plugin_name]()
                        loadables = plugin.examine([filenames])
                        print('loading: {}, {} with {}'.format(series_description, series_number, plugin_name))
                        if loadables and not loadables[0].warning:
                            slicer.app.processEvents()
                            keywords_dict = dict()
                            print('loading:', series_description)
                            volume = plugin.load(loadables[0])
                            if volume is not None:
                                volume_name = series_description + '_' + series_number
                                volume_name = volume_name.replace('/', ' ')
                                volume.SetName(volume_name)
                                ####
                                slicer.util.saveNode(volume, os.path.join(output_study_folder, volume_name + '.nii.gz'))
                                slicer.mrmlScene.RemoveNode(volume)
                                DICOMLib.DICOMCommand("dcm2xml", [filenames[0], os.path.join(output_study_folder,
                                                                                             volume_name + '.xml')]).start()
                                slicer.app.processEvents()
                        else:
                            erronous_patients_t2.append(patient_folder)
                            print('ERRRRRROOOOOR: plugin failed to interpret this series, patient:{}'.format(patient_folder))
                except:
                    pass

----------------------------------------------------------------------------------------------------
ProstateDx-02-0005
----------------------------------------------------------------------------------------------------
loading: T2W_TSE_AX, 401 with DICOMScalarVolumePlugin
('loading:', u'T2W_TSE_AX')
Loading with imageIOName: GDCM
Window/level found in DICOM tags (center=1027.0, width=1785.0) has been applied to volume Unnamed Series
('running: ', u'/home/sedghi/sources/Slicer-4.11.0-2019-02-19-linux-amd64/bin/../bin/dcm2xml', [u'/home/sedghi/Desktop/PROSTATE-DIAGNOSIS/ProstateDx-02-0005/12-13-2005-MRI PROSTATE WITH AND WITHOUT CONTRAST-34346/401-T2WTSEAX-85682/000031.dcm', u'/home/sedghi/Desktop/PROSTATE-DIAGNOSIS_nrrd/ProstateDx-02-0005/1.3.6.1.4.1.14519.5.2.1.4793.2002.136841246334736805333667834346/T2W_TSE_AX_401.xml'])
stopping DICOM process
----------------------------------------------------------------------------------------------------
ProstateDx-02-0004
--------------------

In [20]:
print(erronous_patients_t2)

In [21]:
dicom_root = os.path.join(S.raw_folder,"MRI")
output_folder = os.path.join(S.raw_folder, "dicom2nrrd")
patient_folders = os.listdir(dicom_root)

erronous_patients = []

if not os.path.exists(output_folder):
    os.makedirs(output_folder)

for patient_folder in sorted(patient_folders, reverse=True):
    print('-' * 100)
    print(patient_folder)
    print('-' * 100)
    patient_folder_path = os.path.join(dicom_root, patient_folder)
    output_patient_folder = os.path.join(output_folder, patient_folder)
    if not os.path.isdir(output_patient_folder):
        os.mkdir(output_patient_folder)
    dicomDatabase = TemporaryDatabase('/home/sedghi/tmp')
    indexer = ctk.ctkDICOMIndexer()
    indexer.addDirectory(dicomDatabase, patient_folder_path)
    patients = dicomDatabase.patients()
    for patient in patients:
        for study in dicomDatabase.studiesForPatient(patient):
            output_study_folder = os.path.join(output_patient_folder, study)
            if not os.path.isdir(output_study_folder):
                os.mkdir(output_study_folder)
            series = dicomDatabase.seriesForStudy(study)
            for seriesIndex, currentSeries in enumerate(series, start=1):
                filenames = dicomDatabase.filesForSeries(currentSeries)
                series_description = dicomDatabase.fileValue(filenames[0], "0008,103E")
                series_number = dicomDatabase.fileValue(filenames[0], "0020,0011")
                if ('DWI' in str(series_description)) & (not any(item.upper() in str(series_description).upper() for item in T2_not_desired)):
                    plugin_name = 'DICOMScalarVolumePlugin'
                    plugin = slicer.modules.dicomPlugins[plugin_name]()
                    loadables = plugin.examine([filenames])
                    print('loading: {}, {} with {}'.format(series_description, series_number, plugin_name))
                    if loadables and not loadables[0].warning:
                        slicer.app.processEvents()
                        keywords_dict = dict()
                        print('loading:', series_description)
                        volume = plugin.load(loadables[0])
                        if volume is not None:
                            volume_name = series_description + '_' + series_number
                            volume_name = volume_name.replace('/', ' ')
                            volume.SetName(volume_name)
                            ####
                            slicer.util.saveNode(volume, os.path.join(output_study_folder, volume_name + '.nii.gz'))
                            slicer.mrmlScene.RemoveNode(volume)
                            DICOMLib.DICOMCommand("dcm2xml", [filenames[0], os.path.join(output_study_folder,
                                                                                         volume_name + '.xml')]).start()
                            slicer.app.processEvents()
                    else:
                        erronous_patients.append(patient_folder)
                        print('ERRRRRROOOOOR: plugin failed to interpret this series, patient:{}'.format(patient_folder))

In [22]:
print(erronous_patients)

In [ ]:
import os
import shutil
import warnings
import DICOMLib
import ctk
warnings.filterwarnings('error')
class TemporaryDatabase(ctk.ctkDICOMDatabase):
    @property
    def directory(self):
        return self._directory
    def __init__(self, path):
        ctk.ctkDICOMDatabase.__init__(self)
        self._directory = path
        self.reset()
        self.openDatabase(os.path.join(self.directory, "ctkDICOM.sql"), "mpReviewPreprocessor")
        self.initializeDatabase()
    def reset(self):
        if os.access(self.directory, os.F_OK):
            shutil.rmtree(self.directory)
            os.mkdir(self.directory)
dicom_root = "C://path to your dicom root that has MRIs for all patients"
output_folder = "output folder path that you want to create"
patient_folders = os.listdir(dicom_root)
if not os.path.exists(output_folder):
    os.makedirs(output_folder)
erronous_patients_dce = []
for patient_folder in sorted(patient_folders, reverse=True):
    print('-' * 100)
    print(patient_folder)
    print('-' * 100)
    patient_folder_path = os.path.join(dicom_root, patient_folder)
    output_patient_folder = os.path.join(output_folder, patient_folder)
    if not os.path.isdir(output_patient_folder):
        os.mkdir(output_patient_folder)
    dicomDatabase = TemporaryDatabase('/home/sedghi/tmp')
    indexer = ctk.ctkDICOMIndexer()
    indexer.addDirectory(dicomDatabase, patient_folder_path)
    patients = dicomDatabase.patients()
    for patient in patients:
        for study in dicomDatabase.studiesForPatient(patient):
            output_study_folder = os.path.join(output_patient_folder, study)
            if not os.path.isdir(output_study_folder):
                os.mkdir(output_study_folder)
            series = dicomDatabase.seriesForStudy(study)
            for seriesIndex, currentSeries in enumerate(series, start=1):
                filenames = dicomDatabase.filesForSeries(currentSeries)
                series_description = dicomDatabase.fileValue(filenames[0], "0008,103E")
                series_number = dicomDatabase.fileValue(filenames[0], "0020,0011")
                if len(filenames)>100:
                    print("using the Multivolume Importer")
                    plugin_name = 'MultiVolumeImporterPlugin'
                    plugin = slicer.modules.dicomPlugins[plugin_name]()
                    loadables = plugin.examine([filenames])
                    print('loading: {}, {} with {}'.format(series_description, series_number, plugin_name))
                    if loadables and not loadables[0].warning:
                        slicer.app.processEvents()
                        keywords_dict = dict()
                        print('loading:', series_description)
                        volume = plugin.load(loadables[0])
                        if volume is not None:
                            volume_name = series_description + '_' + series_number
                            volume_name = volume_name.replace('/', ' ')
                            volume.SetName(volume_name)
                            ####
                            slicer.util.saveNode(volume, os.path.join(output_study_folder, volume_name + '.nii.gz'))
                            slicer.mrmlScene.RemoveNode(volume)
                            DICOMLib.DICOMCommand("dcm2xml", [filenames[0], os.path.join(output_study_folder,
                                                                                         volume_name + '.xml')]).start()
                            slicer.app.processEvents()
                    else:
                        erronous_patients_dce.append(patient_folder)
                        print('ERRRRRROOOOOR: plugin failed to interpret this series, patient:{}'.format(patient_folder))
print("there was error for the following cases: {}".format(erronous_patients_dce))